# Frequency Modulation

Frequency modulation is changing the frequency of a carrier according to a modulating signal. If you look on Wikiepedia, [you can find the math behind frequency modulation](https://en.wikipedia.org/wiki/Frequency_modulation)

$$ y(t) = A_c cos \biggl(2\pi f_c t + \frac{A_m f_\Delta}{f_m} sin \big(2\pi f_m t \big)\biggr) $$

Where

+ \\(A_c\\) is the amplitude of the carrier. In this demo, we will assume that is \\(A_c = 1.0\\). However, although we don't do it here, if you are mixing the frequency modulated carrier with other signals you can change this for mixing purposes.

+ \\(A_m\\) is the amplitude of the modulating waveform.

+ \\(f_c\\) is the frequency of the carrier in hertz.

+ \\(f_m\\) is the frequency of the modulator in hertz.

+ \\(f_\Delta\\) is the peak deviation.

+ \\(t\\) is the time in seconds.

As usual with our NumPy sound calculations, we will compute this in discrete samples as:

$$ y(t) = A_c cos \Biggl(\frac{2\pi f_c t_i}{f_s} + \frac{A_m f_\Delta}{f_m} sin \bigg(\frac{2\pi f_m t_i}{f_s} \bigg)\Biggr) $$

All the variables above still apply, with the following changes.

+ \\(t_i\\) is the sample number, which is an integer.

+ \\(f_s\\) is the sampling rate in units of samples per second.

### Yikes that is a lot of math!

Luckily we can simplify it! Let's make a **deviation coefficient** and call it \\(K\\):

$$ K = \frac{A_m f_\Delta}{f_m} $$

The higher you set \\(K\\), the more effect the modulator will have on the waveform.

### Finally, this is what we will implement

$$ y(t) = A_c cos \Biggl(\frac{2\pi f_c t_i}{f_s} + K sin \bigg(\frac{2\pi f_m t_i}{f_s} \bigg)\Biggr) $$

And we will even assume that \\(A_c = 1\\)

## FM to `sounddevice` and wave file, simplifed coefficient

Import libraries

In [1]:
import numpy as np
from scipy.io.wavfile import write
import sounddevice as sd
import time

These variables control the synthesized waveform

+ `f_s`: Sample rate as samples per second.
+ `duration_s`: Duration of the waveform in seconds.
+ `f_c`: Frequency of the carrier, in hz
+ `f_m`: Frequency of the modulator, in hz
+ `k`: deviation constant. No units

In [2]:
f_s = 44100
duration_s = 2.5
f_c = 440.0
f_m = 110.0
k = 0.0

These lines synthesize the waveform

+ `tis`: The sample number as integers
+ `carrier`: The samples of the carrier
+ `modulator`: The samples of the modulating waveform
+ `waveform`: The synthesized waveform

In [3]:
tis = np.arange(duration_s * f_s)
carrier = 2 * np.pi * tis * f_c / tis
modulator = k * np.sin(2 * np.pi * tis * f_m / f_s)
waveform = np.cos(carrier + modulator)

/Users/alicia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in true_divide
  
/Users/alicia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


Write the syntheseized file.

+ `waveform_quiet`: An attenuated waveform that has reasonable volume.

In [4]:
waveform_quiet = waveform * 0.3
waveform_integers = np.int16(waveform_quiet * 32767)
write('fm-out.wav', f_s, waveform_integers)

Play the final waveform out the speakers

In [5]:
sd.play(waveform_quiet, f_s)
time.sleep(duration_s)
sd.stop()